In [1]:
import tensorflow._api.v2.compat.v1 as tf
import tensorflow_hub as hub
import numpy as np
import pickle 

tf.disable_v2_behavior()
tf.reset_default_graph()
tf.logging.set_verbosity(tf.logging.ERROR)

WILDCARD_CHAR = '\t'
MAX_LEN = 16
CHARMAP_PATH = '../DATA/AE_char_map.pickle'
MODEL_PATH = '../DATA/AE_based_Noise_a4_ls128_hn4_al8/'

2023-11-04 23:38:05.639631: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
template = 'p***r'
template = template.replace('*', WILDCARD_CHAR)
template

'p\t\t\tr'

In [3]:
with open(CHARMAP_PATH, 'rb') as f:
    char2id: dict[str, int] = pickle.load(f)
len(char2id)

213

In [4]:
id2char = [''] * len(char2id)
for char, id in char2id.items():
    id2char[id] = char
# id2char

In [5]:
template_as_vector = np.zeros(MAX_LEN, dtype=np.int32)
for i, c in enumerate(template): 
    template_as_vector[i] = char2id.get(c, -1)
template_as_vector = template_as_vector[None, :]
template_as_vector

array([[10, -1, -1, -1, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=int32)

In [6]:
module = hub.Module(MODEL_PATH)
x_placeholder = tf.placeholder(tf.int32, shape=(None, MAX_LEN))

# number of samples
n = 10000
# standard devation
stddev = 0.6

module_inputs = {'x': x_placeholder, 'stddev': (stddev,), 'n': (n,)}
module_dict = module(module_inputs, as_dict=True, signature='sample_from_latent')
prediction_tensor = module_dict['prediction']

In [7]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    samples = sess.run(prediction_tensor, {x_placeholder: template_as_vector})
samples.shape

2023-11-04 23:38:13.981757: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled


(10000, 16)

In [8]:
samples_as_strings = list(map(
    lambda vec: ''.join(id2char[i] for i in vec if i > 0),
    samples
))
# samples_as_strings

In [9]:
samples_filtered = list(set(filter(
    lambda sample: len(sample) == len(template)
                   and all(sc == tc or tc == WILDCARD_CHAR
                           for sc, tc in zip(sample, template)),
    samples_as_strings
)))
# samples_filtered

In [10]:
len(samples_as_strings), len(samples_filtered)

(10000, 206)

In [11]:
'peter' in samples_filtered

True